### Links for other notebooks
Note: Each notebook's first cell has the summary of that notebook

* Notebook2 DavidNet.ipynb: https://colab.research.google.com/drive/1s3KPa8B-nugUZZID1R-gO6iGddIfJIJH  
DavidNet in Tensorflow  
Val Accuracy: **93.13 in 24 epochs** (Exp: BenchMark with Ammar's LR)

* Notebook3 DavidNet.ipnb: https://colab.research.google.com/drive/15sXfkEh4ptegc-7K9AcUVIBwoPQP3Jj_  
Davidnet (10 experiments)  
Val Accuracy: **93.23 in 27 epochs** (Exp: Test for 27 epochs)   

* Notebook4 DavidNet.ipynb: https://colab.research.google.com/drive/1PbkR1_cgtqTSeCqf8Ys6o0HtWuI7T4rx  
DavidNet implementation in Keras (21 experiments)  
Val Accuracy: 89.60 in 24 epochs (Exp: Exp 9-17, 1st best)

* Notebook5 ResNet.ipynb: https://colab.research.google.com/drive/1KkkaZft25mJ9ncHQSAlX50CS_p-ASTdn  
ResNet in Tensorflow keras (2 experiments)  
Val Accuracy: 89.29 in 24 epochs (Exp: BN16)  

* Notebook6 ResNet.ipynb: https://colab.research.google.com/drive/1rKltNhgPwOA5WwJ8XfL9eyofa73e8GrJ  
ResNet using Tensorflow Keras (12 Experiments)  
Val Accuracy: 88.56 in 23 epochs (Exp: MaxLR(8th))

* Notebook7 ResNet.ipynb: https://colab.research.google.com/drive/1T94mqainInCUQfdWRTOGR6T9ksOllFs2  
ResNet using Keras  
Val Accuracy: 87.4 in 24 epochs (Exp4)

### Summary


This notebook has experiments to train model by removing augentation and later changing it. Then there are experiments to find maxLR for oneCycleLR with gradual drop   
**Model**: DavidNet  
**Benchmark**: Val Accuracy: 93.10 in 24 epochs (Exp: Benchmark)  

In [0]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

  Created wheel for gputil: filename=GPUtil-1.4.0-cp36-none-any.whl size=7411 sha256=b6283c5397de406042e219e2c896155d0cde31d0c35c77332bd5268c352a1aaf
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil
Gen RAM Free: 12.9 GB  | Proc size: 153.9 MB
GPU RAM Free: 11441MB | Used: 0MB | Util   0% | Total 11441MB


### Code



In [0]:
import numpy as np
import time, math
from tqdm import tqdm_notebook as tqdm

import tensorflow as tf
import tensorflow.contrib.eager as tfe

In [0]:
tf.enable_eager_execution()


In [0]:
BATCH_SIZE = 512 #@param {type:"integer"}
MOMENTUM = 0.9 #@param {type:"number"}
LEARNING_RATE = 0.4 #@param {type:"number"}
WEIGHT_DECAY = 5e-4 #@param {type:"number"}
EPOCHS = 24 #@param {type:"integer"}

In [0]:
def init_pytorch(shape, dtype=tf.float32, partition_info=None):
  fan = np.prod(shape[:-1])
  bound = 1 / math.sqrt(fan)
  return tf.random.uniform(shape, minval=-bound, maxval=bound, dtype=dtype)

In [0]:
class Conv(tf.keras.Model):
  def __init__(self, c_out):
    super().__init__()
    self.conv = tf.keras.layers.Conv2D(filters=c_out, kernel_size=3, padding="SAME", kernel_initializer=init_pytorch, use_bias=False)    

  def call(self, inputs):
    return tf.nn.relu(self.conv(inputs))

In [0]:
class ConvBN(tf.keras.Model):
  def __init__(self, c_out):
    super().__init__()
    self.conv = tf.keras.layers.Conv2D(filters=c_out, kernel_size=3, padding="SAME", kernel_initializer=init_pytorch, use_bias=False)
    self.bn = tf.keras.layers.BatchNormalization(momentum=0.9, epsilon=1e-5)

  def call(self, inputs):
    return tf.nn.relu(self.bn(self.conv(inputs)))

In [0]:
class ResBlk(tf.keras.Model):
  def __init__(self, c_out, pool, res = False):
    super().__init__()
    self.conv_bn = ConvBN(c_out)
    self.pool = pool
    self.res = res
    if self.res:
      self.res1 = ConvBN(c_out)
      self.res2 = ConvBN(c_out)

  def call(self, inputs):
    h = self.pool(self.conv_bn(inputs))
    if self.res:
      h = h + self.res2(self.res1(h))
    return h

In [0]:
class DavidNet(tf.keras.Model):
  def __init__(self, c=64, weight=0.125):
    super().__init__()
    pool = tf.keras.layers.MaxPooling2D()
    self.init_conv_bn = Conv(c)
    self.blk1 = ResBlk(c*2, pool, res = True)
    self.blk2 = ResBlk(c*4, pool)
    self.blk3 = ResBlk(c*8, pool, res = True)
    self.blk4 = ResBlk(c*16, pool, res = True)
    self.pool = tf.keras.layers.GlobalMaxPool2D()
    self.linear = tf.keras.layers.Dense(10, kernel_initializer=init_pytorch, use_bias=False)
    self.weight = weight

  def call(self, x, y):
    h = self.pool(self.blk4(self.blk3(self.blk2(self.blk1(self.init_conv_bn(x))))))
    h = self.linear(h) * self.weight
    ce = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=h, labels=y)
    loss = tf.reduce_sum(ce)
    correct = tf.reduce_sum(tf.cast(tf.math.equal(tf.argmax(h, axis = 1), y), tf.float32))
    return loss, correct

In [0]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
len_train, len_test = len(x_train), len(x_test)
y_train = y_train.astype('int64').reshape(len_train)
y_test = y_test.astype('int64').reshape(len_test)

train_mean = np.mean(x_train, axis=(0,1,2))
train_std = np.std(x_train, axis=(0,1,2))

test_mean = np.mean(x_train, axis=(0,1,2))
test_std = np.std(x_train, axis=(0,1,2))

normalize = lambda x: ((x - train_mean) / train_std).astype('float32') # todo: check here
normalize_test = lambda x: ((x - test_mean) / test_std).astype('float32') # todo: check here
pad4 = lambda x: np.pad(x, [(0, 0), (4, 4), (4, 4), (0, 0)], mode='reflect')

x_train = normalize(pad4(x_train))
# x_test = normalize(x_test)
x_test = normalize_test(x_test)

170500096/170498071 [==============================] - 12s 0us/step


In [0]:
!wget "https://raw.githubusercontent.com/yu4u/cutout-random-erasing/master/random_eraser.py"
from random_eraser import get_random_eraser
# eraser = get_random_eraser(s_l=0.25, s_h=0.25,v_l=0, v_h=1)
eraser = get_random_eraser(p=0.3,s_l=0.2,s_h=0.2,v_l=0, v_h=1,pixel_level=True)

In [0]:

def augment(images, 
            resize=None, # (width, height) tuple or None
            horizontal_flip=False,
            vertical_flip=False,
            rotate=0, # Maximum rotation angle in degrees
            crop_probability=0, # How often we do crops
            crop_min_percent=0.6, # Minimum linear dimension of a crop
            crop_max_percent=1.,  # Maximum linear dimension of a crop
            mixup=0):  # Mixup coeffecient, see https://arxiv.org/abs/1710.09412.pdf
  if resize is not None:
    images = tf.image.resize_bilinear(images, resize)
  
  # My experiments showed that casting on GPU improves training performance
  if images.dtype is not tf.float32:
    images = tf.image.convert_image_dtype(images, dtype=tf.float32)
    images = tf.subtract(images, 0.5)
    images = tf.multiply(images, 2.0)
  # labels = tf.to_float(labels)

  with tf.name_scope('augmentation'):
    shp = tf.shape(images)
    batch_size, height, width = shp[0], shp[1], shp[2]
    width = tf.cast(width, tf.float32)
    height = tf.cast(height, tf.float32)

    # The list of affine transformations that our image will go under.
    # Every element is Nx8 tensor, where N is a batch size.
    transforms = []
    identity = tf.constant([1, 0, 0, 0, 1, 0, 0, 0], dtype=tf.float32)
    if horizontal_flip:
      coin = tf.less(tf.random_uniform([batch_size], 0, 1.0), 0.5)
      flip_transform = tf.convert_to_tensor(
          [-1., 0., width, 0., 1., 0., 0., 0.], dtype=tf.float32)
      transforms.append(
          tf.where(coin,
                   tf.tile(tf.expand_dims(flip_transform, 0), [batch_size, 1]),
                   tf.tile(tf.expand_dims(identity, 0), [batch_size, 1])))

    if vertical_flip:
      coin = tf.less(tf.random_uniform([batch_size], 0, 1.0), 0.5)
      flip_transform = tf.convert_to_tensor(
          [1, 0, 0, 0, -1, height, 0, 0], dtype=tf.float32)
      transforms.append(
          tf.where(coin,
                   tf.tile(tf.expand_dims(flip_transform, 0), [batch_size, 1]),
                   tf.tile(tf.expand_dims(identity, 0), [batch_size, 1])))

    if rotate > 0:
      angle_rad = rotate / 180 * math.pi
      angles = tf.random_uniform([batch_size], -angle_rad, angle_rad)
      transforms.append(
          tf.contrib.image.angles_to_projective_transforms(
              angles, height, width))

    if crop_probability > 0:
      crop_pct = tf.random_uniform([batch_size], crop_min_percent,
                                   crop_max_percent)
      left = tf.random_uniform([batch_size], 0, width * (1 - crop_pct))
      top = tf.random_uniform([batch_size], 0, height * (1 - crop_pct))
      crop_transform = tf.stack([
          crop_pct,
          tf.zeros([batch_size]), top,
          tf.zeros([batch_size]), crop_pct, left,
          tf.zeros([batch_size]),
          tf.zeros([batch_size])
      ], 1)

      coin = tf.less(
          tf.random_uniform([batch_size], 0, 1.0), crop_probability)
      transforms.append(
          tf.where(coin, crop_transform,
                   tf.tile(tf.expand_dims(identity, 0), [batch_size, 1])))

    if transforms:
      images = tf.contrib.image.transform(
          images,
          tf.contrib.image.compose_transforms(*transforms),
          interpolation='BILINEAR') # or 'NEAREST'

    def cshift(values): # Circular shift in batch dimension
      return tf.concat([values[-1:, ...], values[:-1, ...]], 0)

    if mixup > 0:
      mixup = 1.0 * mixup # Convert to float, as tf.distributions.Beta requires floats.
      beta = tf.distributions.Beta(mixup, mixup)
      lam = beta.sample(batch_size)
      ll = tf.expand_dims(tf.expand_dims(tf.expand_dims(lam, -1), -1), -1)
      images = ll * images + (1 - ll) * cshift(images)
      # labels = lam * labels + (1 - lam) * cshift(labels)

  return images

In [0]:
x_train = augment(images=x_train, 
                          rotate=10)#, crop_probability=0.8, )

In [0]:
model = DavidNet()
batches_per_epoch = len_train//BATCH_SIZE + 1

# lr_schedule = lambda t: np.interp([t], [0, (EPOCHS+1)//5, EPOCHS], [0, LEARNING_RATE, 0])[0]
lr_schedule = lambda t: np.interp([t], [0, (EPOCHS+1)//5, int(0.8*EPOCHS), EPOCHS], [0, LEARNING_RATE, 0.1*LEARNING_RATE, 0.005])[0]
global_step = tf.train.get_or_create_global_step()
lr_func = lambda: lr_schedule(global_step/batches_per_epoch)/BATCH_SIZE
opt = tf.train.MomentumOptimizer(lr_func, momentum=MOMENTUM, use_nesterov=True)
# data_aug = lambda x, y: (tf.image.random_flip_left_right(tf.random_crop(x, [32, 32, 3])), y)
data_aug = lambda x, y: tf.switch_case(
        tf.random.uniform([],0,1,tf.dtypes.int32), 
        branch_fns={
            0: lambda: (tf.image.random_flip_left_right(tf.random_crop(x, [32, 32, 3])), y),
            # 1: lambda: (tf.image.central_crop(x, 0.8), y),
            1: lambda: (tf.contrib.image.rotate(tf.random_crop(x, [32, 32, 3]), np.random.randint(low=1,high=15) * math.pi / 180, interpolation='BILINEAR'),y)
            # 1: lambda: 
        }, 
        # default=lambda: (tf.random_crop(x, [32, 32, 3]), y)
        default=lambda: (tf.image.random_flip_left_right(tf.random_crop(x, [32, 32, 3])), y)
    )

### BenchMark

In [0]:
t = time.time()
test_set = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(BATCH_SIZE)

for epoch in range(EPOCHS):
  train_loss = test_loss = train_acc = test_acc = 0.0
  train_set = tf.data.Dataset.from_tensor_slices((x_train, y_train)).map(data_aug).shuffle(len_train).batch(BATCH_SIZE).prefetch(1)

  tf.keras.backend.set_learning_phase(1)
  for (x, y) in tqdm(train_set):
    with tf.GradientTape() as tape:
      loss, correct = model(x, y)

    var = model.trainable_variables
    grads = tape.gradient(loss, var)
    for g, v in zip(grads, var):
      g += v * WEIGHT_DECAY * BATCH_SIZE
    opt.apply_gradients(zip(grads, var), global_step=global_step)

    train_loss += loss.numpy()
    train_acc += correct.numpy()

  tf.keras.backend.set_learning_phase(0)
  for (x, y) in test_set:
    loss, correct = model(x, y)
    test_loss += loss.numpy()
    test_acc += correct.numpy()
    
  print('epoch:', epoch+1, 'lr:', lr_schedule(epoch+1), 'train loss:', train_loss / len_train, 'train acc:', train_acc / len_train, 'val loss:', test_loss / len_test, 'val acc:', test_acc / len_test, 'time:', time.time() - t)


epoch: 1 lr: 0.08 train loss: 1.5673736126708984 train acc: 0.43722 val loss: 1.2368145568847657 val acc: 0.5492 time: 80.57430982589722



epoch: 2 lr: 0.16 train loss: 0.8291827587890624 train acc: 0.707 val loss: 0.8341683685302734 val acc: 0.7114 time: 147.37661361694336



epoch: 3 lr: 0.24 train loss: 0.6343590902709961 train acc: 0.78006 val loss: 0.7416124740600586 val acc: 0.7599 time: 214.04439735412598



epoch: 4 lr: 0.32 train loss: 0.5407362075805664 train acc: 0.81262 val loss: 0.6011689559936524 val acc: 0.7929 time: 280.59751987457275



epoch: 5 lr: 0.4 train loss: 0.47634036224365234 train acc: 0.8379 val loss: 0.5181582809448242 val acc: 0.8229 time: 347.53501439094543


...


epoch: 20 lr: 0.08421052631578951 train loss: 0.03810818576812744 train acc: 0.9889 val loss: 0.257457247543335 val acc: 0.9248 time: 1349.2928335666656



epoch: 21 lr: 0.06315789473684214 train loss: 0.031118259716033936 train acc: 0.99108 val loss: 0.2638925582885742 val acc: 0.9263 time: 1416.204303741455



epoch: 22 lr: 0.04210526315789476 train loss: 0.025494536838531492 train acc: 0.99346 val loss: 0.2495221580505371 val acc: 0.9302 time: 1483.0338776111603



epoch: 23 lr: 0.02105263157894738 train loss: 0.022272464170455933 train acc: 0.9945 val loss: 0.24605972785949706 val acc: 0.9307 time: 1549.8490784168243



epoch: 24 lr: 0.0 train loss: 0.02078292353630066 train acc: 0.99498 val loss: 0.2454392734527588 val acc: 0.931 time: 1616.5044195652008


### BenchMark without extra BN

In [0]:
t = time.time()
test_set = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(BATCH_SIZE)

for epoch in range(EPOCHS):
  train_loss = test_loss = train_acc = test_acc = 0.0
  train_set = tf.data.Dataset.from_tensor_slices((x_train, y_train)).map(data_aug).shuffle(len_train).batch(BATCH_SIZE).prefetch(1)

  tf.keras.backend.set_learning_phase(1)
  for (x, y) in tqdm(train_set):
    with tf.GradientTape() as tape:
      loss, correct = model(x, y)

    var = model.trainable_variables
    grads = tape.gradient(loss, var)
    for g, v in zip(grads, var):
      g += v * WEIGHT_DECAY * BATCH_SIZE
    opt.apply_gradients(zip(grads, var), global_step=global_step)

    train_loss += loss.numpy()
    train_acc += correct.numpy()

  tf.keras.backend.set_learning_phase(0)
  for (x, y) in test_set:
    loss, correct = model(x, y)
    test_loss += loss.numpy()
    test_acc += correct.numpy()
    
  print('epoch:', epoch+1, 'lr:', lr_schedule(epoch+1), 'train loss:', train_loss / len_train, 'train acc:', train_acc / len_train, 'val loss:', test_loss / len_test, 'val acc:', test_acc / len_test, 'time:', time.time() - t)


epoch: 1 lr: 0.08 train loss: 1.2859700598144532 train acc: 0.53412 val loss: 0.8336404083251953 val acc: 0.7056 time: 65.72036981582642



epoch: 2 lr: 0.16 train loss: 0.7266783175659179 train acc: 0.74418 val loss: 0.715204833984375 val acc: 0.7535 time: 131.0114870071411



epoch: 3 lr: 0.24 train loss: 0.5749895935058593 train acc: 0.80072 val loss: 0.7324721160888672 val acc: 0.7484 time: 196.2042441368103



epoch: 4 lr: 0.32 train loss: 0.4941643084716797 train acc: 0.83242 val loss: 0.5341999465942383 val acc: 0.8211 time: 261.5650517940521



epoch: 5 lr: 0.4 train loss: 0.38463194931030276 train acc: 0.86714 val loss: 0.630856558227539 val acc: 0.7866 time: 326.8199689388275


...


epoch: 20 lr: 0.032200000000000006 train loss: 0.03129239737510681 train acc: 0.99146 val loss: 0.2551196815490723 val acc: 0.9242 time: 1305.6084349155426



epoch: 21 lr: 0.024400000000000005 train loss: 0.03132392505645752 train acc: 0.99084 val loss: 0.25356032028198244 val acc: 0.9241 time: 1370.7649157047272



epoch: 22 lr: 0.016600000000000004 train loss: 0.03001959012031555 train acc: 0.9915 val loss: 0.25228773803710935 val acc: 0.924 time: 1436.0178806781769



epoch: 23 lr: 0.008800000000000002 train loss: 0.027385428304672242 train acc: 0.9928 val loss: 0.2522149917602539 val acc: 0.9239 time: 1501.4287130832672



epoch: 24 lr: 0.001 train loss: 0.02721629337310791 train acc: 0.99258 val loss: 0.2514957916259766 val acc: 0.9239 time: 1566.635277748108


### BenchMark without Augmentation and First BN : Train reached 100

In [0]:
t = time.time()
test_set = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(BATCH_SIZE)

for epoch in range(EPOCHS):
  train_loss = test_loss = train_acc = test_acc = 0.0
  # train_set = tf.data.Dataset.from_tensor_slices((x_train, y_train)).map(data_aug).shuffle(len_train).batch(BATCH_SIZE).prefetch(1)
  train_set = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(BATCH_SIZE).prefetch(1)

  tf.keras.backend.set_learning_phase(1)
  for (x, y) in tqdm(train_set):
    with tf.GradientTape() as tape:
      loss, correct = model(x, y)

    var = model.trainable_variables
    grads = tape.gradient(loss, var)
    for g, v in zip(grads, var):
      g += v * WEIGHT_DECAY * BATCH_SIZE
    opt.apply_gradients(zip(grads, var), global_step=global_step)

    train_loss += loss.numpy()
    train_acc += correct.numpy()

  tf.keras.backend.set_learning_phase(0)
  for (x, y) in test_set:
    loss, correct = model(x, y)
    test_loss += loss.numpy()
    test_acc += correct.numpy()
    
  print('epoch:', epoch+1, 'lr:', lr_schedule(epoch+1), 'train loss:', train_loss / len_train, 'train acc:', train_acc / len_train, 'val loss:', test_loss / len_test, 'val acc:', test_acc / len_test, 'time:', time.time() - t)


epoch: 1 lr: 0.08 train loss: 1.5673690911865235 train acc: 0.43916 val loss: 1.1392370941162109 val acc: 0.5959 time: 111.104576587677



epoch: 2 lr: 0.16 train loss: 0.7960348654174805 train acc: 0.71598 val loss: 0.969080810546875 val acc: 0.6655 time: 204.68000388145447



epoch: 3 lr: 0.24 train loss: 0.5750762902832032 train acc: 0.80074 val loss: 0.872841079711914 val acc: 0.7085 time: 298.27933716773987



epoch: 4 lr: 0.32 train loss: 0.4318459466552734 train acc: 0.8538 val loss: 0.7656824066162109 val acc: 0.7439 time: 391.9845116138458



epoch: 5 lr: 0.4 train loss: 0.3335385580444336 train acc: 0.88706 val loss: 1.1862882720947265 val acc: 0.6745 time: 485.656131029129


...


epoch: 17 lr: 0.09142857142857147 train loss: 0.00033664067178964613 train acc: 1.0 val loss: 0.5855786254882812 val acc: 0.8483 time: 1609.3777675628662



epoch: 18 lr: 0.06571428571428573 train loss: 0.00032057932317256925 train acc: 1.0 val loss: 0.5865876449584961 val acc: 0.8488 time: 1703.1454510688782



epoch: 19 lr: 0.04000000000000001 train loss: 0.0003095045708119869 train acc: 1.0 val loss: 0.5872516006469727 val acc: 0.8483 time: 1796.7931394577026



epoch: 20 lr: 0.032200000000000006 train loss: 0.0003024866585433483 train acc: 1.0 val loss: 0.5876671173095703 val acc: 0.8483 time: 1890.5168538093567


KeyboardInterrupt: ignored

### LR with Gradual drop after One-CycleLR

In [0]:
t = time.time()
test_set = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(BATCH_SIZE)

for epoch in range(EPOCHS):
  train_loss = test_loss = train_acc = test_acc = 0.0
  train_set = tf.data.Dataset.from_tensor_slices((x_train, y_train)).map(data_aug).shuffle(len_train).batch(BATCH_SIZE).prefetch(1)

  tf.keras.backend.set_learning_phase(1)
  for (x, y) in tqdm(train_set):
    with tf.GradientTape() as tape:
      loss, correct = model(x, y)

    var = model.trainable_variables
    grads = tape.gradient(loss, var)
    for g, v in zip(grads, var):
      g += v * WEIGHT_DECAY * BATCH_SIZE
    opt.apply_gradients(zip(grads, var), global_step=global_step)

    train_loss += loss.numpy()
    train_acc += correct.numpy()

  tf.keras.backend.set_learning_phase(0)
  for (x, y) in test_set:
    loss, correct = model(x, y)
    test_loss += loss.numpy()
    test_acc += correct.numpy()
    
  print('epoch:', epoch+1, 'lr:', lr_schedule(epoch+1), 'train loss:', train_loss / len_train, 'train acc:', train_acc / len_train, 'val loss:', test_loss / len_test, 'val acc:', test_acc / len_test, 'time:', time.time() - t)

epoch: 1 lr: 0.08 train loss: 1.5781250744628905 train acc: 0.42716 val loss: 1.197358255004883 val acc: 0.5875 time: 67.35015916824341


epoch: 2 lr: 0.16 train loss: 0.849458623046875 train acc: 0.69822 val loss: 1.0340927764892578 val acc: 0.651 time: 130.46793007850647


epoch: 3 lr: 0.24 train loss: 0.6339761224365235 train acc: 0.77728 val loss: 0.7426778518676758 val acc: 0.7439 time: 193.4626920223236


epoch: 4 lr: 0.32 train loss: 0.5422529977416992 train acc: 0.81206 val loss: 0.5425949234008789 val acc: 0.8173 time: 256.4593138694763


epoch: 5 lr: 0.4 train loss: 0.47718792022705075 train acc: 0.83612 val loss: 0.5699813064575195 val acc: 0.8073 time: 319.596755027771


...

epoch: 20 lr: 0.032200000000000006 train loss: 0.03519728136062622 train acc: 0.99034 val loss: 0.24964814338684083 val acc: 0.9288 time: 1265.3090002536774


epoch: 21 lr: 0.024400000000000005 train loss: 0.030772331314086913 train acc: 0.99208 val loss: 0.24758256492614747 val acc: 0.9282 time: 1328.153358221054


epoch: 22 lr: 0.016600000000000004 train loss: 0.027760543994903564 train acc: 0.99316 val loss: 0.24373788299560548 val acc: 0.9311 time: 1391.0234067440033


epoch: 23 lr: 0.008800000000000002 train loss: 0.026477796268463135 train acc: 0.99326 val loss: 0.24229511184692382 val acc: 0.9295 time: 1453.9991524219513


epoch: 24 lr: 0.001 train loss: 0.02582888198852539 train acc: 0.994 val loss: 0.2426891700744629 val acc: 0.9299 time: 1516.8821604251862


### LR with Gradual drop after One-CycleLR with Seperate Test Normalization

In [0]:
t = time.time()
test_set = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(BATCH_SIZE)

for epoch in range(EPOCHS):
  train_loss = test_loss = train_acc = test_acc = 0.0
  train_set = tf.data.Dataset.from_tensor_slices((x_train, y_train)).map(data_aug).shuffle(len_train).batch(BATCH_SIZE).prefetch(1)

  tf.keras.backend.set_learning_phase(1)
  for (x, y) in tqdm(train_set):
    with tf.GradientTape() as tape:
      loss, correct = model(x, y)

    var = model.trainable_variables
    grads = tape.gradient(loss, var)
    for g, v in zip(grads, var):
      g += v * WEIGHT_DECAY * BATCH_SIZE
    opt.apply_gradients(zip(grads, var), global_step=global_step)

    train_loss += loss.numpy()
    train_acc += correct.numpy()

  tf.keras.backend.set_learning_phase(0)
  for (x, y) in test_set:
    loss, correct = model(x, y)
    test_loss += loss.numpy()
    test_acc += correct.numpy()
    
  print('epoch:', epoch+1, 'lr:', lr_schedule(epoch+1), 'train loss:', train_loss / len_train, 'train acc:', train_acc / len_train, 'val loss:', test_loss / len_test, 'val acc:', test_acc / len_test, 'time:', time.time() - t)


epoch: 1 lr: 0.08 train loss: 1.5617154077148439 train acc: 0.43632 val loss: 1.3378051727294922 val acc: 0.5329 time: 73.42414212226868



epoch: 2 lr: 0.16 train loss: 0.836080629272461 train acc: 0.70398 val loss: 1.0398654083251953 val acc: 0.6677 time: 136.92781925201416



epoch: 3 lr: 0.24 train loss: 0.6254481799316406 train acc: 0.78398 val loss: 0.8237745101928711 val acc: 0.737 time: 200.23369097709656



epoch: 4 lr: 0.32 train loss: 0.5398001080322266 train acc: 0.81392 val loss: 0.6408422973632812 val acc: 0.7743 time: 263.4912347793579



epoch: 5 lr: 0.4 train loss: 0.47980604278564454 train acc: 0.8352 val loss: 0.5952761169433594 val acc: 0.8025 time: 326.8753447532654


...


epoch: 20 lr: 0.032200000000000006 train loss: 0.03348892301559448 train acc: 0.99112 val loss: 0.2455391757965088 val acc: 0.9269 time: 1276.899026632309



epoch: 21 lr: 0.024400000000000005 train loss: 0.031195758628845216 train acc: 0.99226 val loss: 0.2497573871612549 val acc: 0.9265 time: 1340.2563495635986



epoch: 22 lr: 0.016600000000000004 train loss: 0.028970465030670165 train acc: 0.99272 val loss: 0.24941670608520508 val acc: 0.9267 time: 1403.6912610530853



epoch: 23 lr: 0.008800000000000002 train loss: 0.027110953369140624 train acc: 0.99354 val loss: 0.24591126403808594 val acc: 0.9281 time: 1466.865451335907



epoch: 24 lr: 0.001 train loss: 0.025880767068862914 train acc: 0.99386 val loss: 0.24678536376953125 val acc: 0.9271 time: 1529.9745862483978


### LR with Gradual drop after One-CycleLR

In [0]:
model = DavidNet()
batches_per_epoch = len_train//BATCH_SIZE + 1

# lr_schedule = lambda t: np.interp([t], [0, (EPOCHS+1)//5, EPOCHS], [0, LEARNING_RATE, 0])[0]
lr_schedule = lambda t: np.interp([t], [0, (EPOCHS+1)//5, int(0.8*EPOCHS), EPOCHS], [0, LEARNING_RATE, 0.15*LEARNING_RATE, 0.01])[0]
global_step = tf.train.get_or_create_global_step()
lr_func = lambda: lr_schedule(global_step/batches_per_epoch)/BATCH_SIZE
opt = tf.train.MomentumOptimizer(lr_func, momentum=MOMENTUM, use_nesterov=True)
data_aug = lambda x, y: (tf.image.random_flip_left_right(tf.random_crop(x, [32, 32, 3])), y)

In [0]:
t = time.time()
test_set = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(BATCH_SIZE)

for epoch in range(EPOCHS):
  train_loss = test_loss = train_acc = test_acc = 0.0
  train_set = tf.data.Dataset.from_tensor_slices((x_train, y_train)).map(data_aug).shuffle(len_train).batch(BATCH_SIZE).prefetch(1)

  tf.keras.backend.set_learning_phase(1)
  for (x, y) in tqdm(train_set):
    with tf.GradientTape() as tape:
      loss, correct = model(x, y)

    var = model.trainable_variables
    grads = tape.gradient(loss, var)
    for g, v in zip(grads, var):
      g += v * WEIGHT_DECAY * BATCH_SIZE
    opt.apply_gradients(zip(grads, var), global_step=global_step)

    train_loss += loss.numpy()
    train_acc += correct.numpy()

  tf.keras.backend.set_learning_phase(0)
  for (x, y) in test_set:
    loss, correct = model(x, y)
    test_loss += loss.numpy()
    test_acc += correct.numpy()
    
  print('epoch:', epoch+1, 'lr:', lr_schedule(epoch+1), 'train loss:', train_loss / len_train, 'train acc:', train_acc / len_train, 'val loss:', test_loss / len_test, 'val acc:', test_acc / len_test, 'time:', time.time() - t)


epoch: 1 lr: 0.08 train loss: 1.572046083984375 train acc: 0.42786 val loss: 1.5202794067382812 val acc: 0.4875 time: 75.9919843673706



epoch: 2 lr: 0.16 train loss: 0.8463304397583008 train acc: 0.69774 val loss: 1.0581017669677735 val acc: 0.6357 time: 138.58160400390625



epoch: 3 lr: 0.24 train loss: 0.6340433840942383 train acc: 0.7789 val loss: 0.8111426498413086 val acc: 0.7285 time: 201.1136486530304



epoch: 4 lr: 0.32 train loss: 0.5395852044677735 train acc: 0.8138 val loss: 0.6576350845336915 val acc: 0.7847 time: 263.77787828445435



epoch: 5 lr: 0.4 train loss: 0.4743790112304688 train acc: 0.83844 val loss: 0.5384391540527343 val acc: 0.8137 time: 326.33546805381775


...


epoch: 20 lr: 0.05 train loss: 0.03579047821044922 train acc: 0.9897 val loss: 0.246568127822876 val acc: 0.9297 time: 1265.9602336883545



epoch: 21 lr: 0.04 train loss: 0.03048294871330261 train acc: 0.99202 val loss: 0.24421445541381837 val acc: 0.9291 time: 1328.51975607872



epoch: 22 lr: 0.030000000000000002 train loss: 0.02842438310623169 train acc: 0.99242 val loss: 0.24273134727478027 val acc: 0.9296 time: 1391.401605129242



epoch: 23 lr: 0.020000000000000004 train loss: 0.024963816080093383 train acc: 0.99368 val loss: 0.24412520751953126 val acc: 0.93 time: 1454.256047964096



epoch: 24 lr: 0.01 train loss: 0.02358230936050415 train acc: 0.99436 val loss: 0.24446614303588868 val acc: 0.9305 time: 1516.957738161087


### LR with Gradual drop after One-CycleLR

In [0]:
model = DavidNet()
batches_per_epoch = len_train//BATCH_SIZE + 1

# lr_schedule = lambda t: np.interp([t], [0, (EPOCHS+1)//5, EPOCHS], [0, LEARNING_RATE, 0])[0]
lr_schedule = lambda t: np.interp([t], [0, (EPOCHS+1)//5, int(0.8*EPOCHS), EPOCHS], [0, LEARNING_RATE, 0.12*LEARNING_RATE, 0.01])[0]
global_step = tf.train.get_or_create_global_step()
lr_func = lambda: lr_schedule(global_step/batches_per_epoch)/BATCH_SIZE
opt = tf.train.MomentumOptimizer(lr_func, momentum=MOMENTUM, use_nesterov=True)
data_aug = lambda x, y: (tf.image.random_flip_left_right(tf.random_crop(x, [32, 32, 3])), y)

In [0]:
t = time.time()
test_set = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(BATCH_SIZE)

for epoch in range(EPOCHS):
  train_loss = test_loss = train_acc = test_acc = 0.0
  train_set = tf.data.Dataset.from_tensor_slices((x_train, y_train)).map(data_aug).shuffle(len_train).batch(BATCH_SIZE).prefetch(1)

  tf.keras.backend.set_learning_phase(1)
  for (x, y) in tqdm(train_set):
    with tf.GradientTape() as tape:
      loss, correct = model(x, y)

    var = model.trainable_variables
    grads = tape.gradient(loss, var)
    for g, v in zip(grads, var):
      g += v * WEIGHT_DECAY * BATCH_SIZE
    opt.apply_gradients(zip(grads, var), global_step=global_step)

    train_loss += loss.numpy()
    train_acc += correct.numpy()

  tf.keras.backend.set_learning_phase(0)
  for (x, y) in test_set:
    loss, correct = model(x, y)
    test_loss += loss.numpy()
    test_acc += correct.numpy()
    
  print('epoch:', epoch+1, 'lr:', lr_schedule(epoch+1), 'train loss:', train_loss / len_train, 'train acc:', train_acc / len_train, 'val loss:', test_loss / len_test, 'val acc:', test_acc / len_test, 'time:', time.time() - t)


epoch: 1 lr: 0.08 train loss: 1.5764755249023437 train acc: 0.43158 val loss: 1.21459326171875 val acc: 0.5723 time: 72.41288375854492



epoch: 2 lr: 0.16 train loss: 0.8421278759765625 train acc: 0.7016 val loss: 0.8049965667724609 val acc: 0.7201 time: 135.02836632728577



epoch: 3 lr: 0.24 train loss: 0.6243019998168945 train acc: 0.78252 val loss: 1.1561366882324218 val acc: 0.648 time: 197.73972821235657



epoch: 4 lr: 0.32 train loss: 0.5557411483764648 train acc: 0.8064 val loss: 0.779678025817871 val acc: 0.748 time: 260.41374683380127



epoch: 5 lr: 0.4 train loss: 0.4701523489379883 train acc: 0.83778 val loss: 0.6792806259155273 val acc: 0.7874 time: 323.30933237075806


...


epoch: 20 lr: 0.0404 train loss: 0.03443793603897095 train acc: 0.99098 val loss: 0.2570962371826172 val acc: 0.9243 time: 1263.8208668231964



epoch: 21 lr: 0.0328 train loss: 0.031042065839767458 train acc: 0.99212 val loss: 0.2589907272338867 val acc: 0.9232 time: 1326.501947402954



epoch: 22 lr: 0.0252 train loss: 0.02840428681373596 train acc: 0.99284 val loss: 0.25457475662231444 val acc: 0.9255 time: 1389.2130670547485



epoch: 23 lr: 0.0176 train loss: 0.026847732057571412 train acc: 0.99312 val loss: 0.2547789939880371 val acc: 0.9257 time: 1451.8105771541595



epoch: 24 lr: 0.01 train loss: 0.024859117164611817 train acc: 0.99394 val loss: 0.25448406867980955 val acc: 0.9263 time: 1514.6610689163208


### LR with Gradual drop after One-CycleLR

In [0]:
model = DavidNet()
batches_per_epoch = len_train//BATCH_SIZE + 1

# lr_schedule = lambda t: np.interp([t], [0, (EPOCHS+1)//5, EPOCHS], [0, LEARNING_RATE, 0])[0]
lr_schedule = lambda t: np.interp([t], [0, (EPOCHS+1)//5, int(0.75*EPOCHS), EPOCHS], [0, LEARNING_RATE, 0.15*LEARNING_RATE, 0.01])[0]
global_step = tf.train.get_or_create_global_step()
lr_func = lambda: lr_schedule(global_step/batches_per_epoch)/BATCH_SIZE
opt = tf.train.MomentumOptimizer(lr_func, momentum=MOMENTUM, use_nesterov=True)
data_aug = lambda x, y: (tf.image.random_flip_left_right(tf.random_crop(x, [32, 32, 3])), y)

In [0]:
t = time.time()
test_set = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(BATCH_SIZE)

for epoch in range(EPOCHS):
  train_loss = test_loss = train_acc = test_acc = 0.0
  train_set = tf.data.Dataset.from_tensor_slices((x_train, y_train)).map(data_aug).shuffle(len_train).batch(BATCH_SIZE).prefetch(1)

  tf.keras.backend.set_learning_phase(1)
  for (x, y) in tqdm(train_set):
    with tf.GradientTape() as tape:
      loss, correct = model(x, y)

    var = model.trainable_variables
    grads = tape.gradient(loss, var)
    for g, v in zip(grads, var):
      g += v * WEIGHT_DECAY * BATCH_SIZE
    opt.apply_gradients(zip(grads, var), global_step=global_step)

    train_loss += loss.numpy()
    train_acc += correct.numpy()

  tf.keras.backend.set_learning_phase(0)
  for (x, y) in test_set:
    loss, correct = model(x, y)
    test_loss += loss.numpy()
    test_acc += correct.numpy()
    
  print('epoch:', epoch+1, 'lr:', lr_schedule(epoch+1), 'train loss:', train_loss / len_train, 'train acc:', train_acc / len_train, 'val loss:', test_loss / len_test, 'val acc:', test_acc / len_test, 'time:', time.time() - t)


epoch: 1 lr: 0.08 train loss: 1.5788017864990234 train acc: 0.42946 val loss: 1.203389111328125 val acc: 0.5766 time: 72.36773324012756



epoch: 2 lr: 0.16 train loss: 0.8321050769042969 train acc: 0.70628 val loss: 1.0963363861083983 val acc: 0.6494 time: 135.07237148284912



epoch: 3 lr: 0.24 train loss: 0.6357986819458008 train acc: 0.777 val loss: 0.8308140075683593 val acc: 0.7331 time: 197.55912518501282



epoch: 4 lr: 0.32 train loss: 0.5418064562988282 train acc: 0.81316 val loss: 1.0762475250244141 val acc: 0.6544 time: 260.1972095966339



epoch: 5 lr: 0.4 train loss: 0.4761518014526367 train acc: 0.83748 val loss: 0.8346518798828125 val acc: 0.74 time: 322.62034583091736


...


epoch: 20 lr: 0.043333333333333335 train loss: 0.03665470775604248 train acc: 0.98978 val loss: 0.24693914108276369 val acc: 0.9272 time: 1262.0146005153656



epoch: 21 lr: 0.034999999999999996 train loss: 0.03211457399368286 train acc: 0.99152 val loss: 0.2462463653564453 val acc: 0.9282 time: 1324.5957052707672



epoch: 22 lr: 0.026666666666666665 train loss: 0.03030180227279663 train acc: 0.99172 val loss: 0.24153601722717286 val acc: 0.9294 time: 1387.3312833309174



epoch: 23 lr: 0.018333333333333333 train loss: 0.02680439739227295 train acc: 0.99302 val loss: 0.24117785148620605 val acc: 0.9294 time: 1449.7635388374329



epoch: 24 lr: 0.01 train loss: 0.024386605606079102 train acc: 0.99454 val loss: 0.24342081756591796 val acc: 0.929 time: 1512.3256363868713


### LR with Gradual drop after One-CycleLR

In [0]:
model = DavidNet()
batches_per_epoch = len_train//BATCH_SIZE + 1

# lr_schedule = lambda t: np.interp([t], [0, (EPOCHS+1)//5, EPOCHS], [0, LEARNING_RATE, 0])[0]
lr_schedule = lambda t: np.interp([t], [0, (EPOCHS+1)//5, int(0.8*EPOCHS), EPOCHS], [0, LEARNING_RATE, 0.1*LEARNING_RATE, 0])[0]
global_step = tf.train.get_or_create_global_step()
lr_func = lambda: lr_schedule(global_step/batches_per_epoch)/BATCH_SIZE
opt = tf.train.MomentumOptimizer(lr_func, momentum=MOMENTUM, use_nesterov=True)
data_aug = lambda x, y: (tf.image.random_flip_left_right(tf.random_crop(x, [32, 32, 3])), y)

In [0]:
t = time.time()
test_set = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(BATCH_SIZE)

for epoch in range(EPOCHS):
  train_loss = test_loss = train_acc = test_acc = 0.0
  train_set = tf.data.Dataset.from_tensor_slices((x_train, y_train)).map(data_aug).shuffle(len_train).batch(BATCH_SIZE).prefetch(1)

  tf.keras.backend.set_learning_phase(1)
  for (x, y) in tqdm(train_set):
    with tf.GradientTape() as tape:
      loss, correct = model(x, y)

    var = model.trainable_variables
    grads = tape.gradient(loss, var)
    for g, v in zip(grads, var):
      g += v * WEIGHT_DECAY * BATCH_SIZE
    opt.apply_gradients(zip(grads, var), global_step=global_step)

    train_loss += loss.numpy()
    train_acc += correct.numpy()

  tf.keras.backend.set_learning_phase(0)
  for (x, y) in test_set:
    loss, correct = model(x, y)
    test_loss += loss.numpy()
    test_acc += correct.numpy()
    
  print('epoch:', epoch+1, 'lr:', lr_schedule(epoch+1), 'train loss:', train_loss / len_train, 'train acc:', train_acc / len_train, 'val loss:', test_loss / len_test, 'val acc:', test_acc / len_test, 'time:', time.time() - t)


epoch: 1 lr: 0.08 train loss: 1.5729883239746094 train acc: 0.4318 val loss: 1.4432714904785156 val acc: 0.4921 time: 76.9434449672699



epoch: 2 lr: 0.16 train loss: 0.8328930831909179 train acc: 0.70444 val loss: 0.7488742691040039 val acc: 0.7399 time: 141.28321814537048



epoch: 3 lr: 0.24 train loss: 0.6374128131103516 train acc: 0.77728 val loss: 0.7638113677978515 val acc: 0.7594 time: 205.80347967147827



epoch: 4 lr: 0.32 train loss: 0.5316774468994141 train acc: 0.81504 val loss: 0.7109573867797851 val acc: 0.7683 time: 270.28719830513



epoch: 5 lr: 0.4 train loss: 0.4898508883666992 train acc: 0.83256 val loss: 0.5315122955322266 val acc: 0.8213 time: 334.8328628540039


...


epoch: 20 lr: 0.03200000000000001 train loss: 0.03466308523178101 train acc: 0.99022 val loss: 0.24620797805786132 val acc: 0.9287 time: 1325.6473724842072



epoch: 21 lr: 0.024000000000000004 train loss: 0.0329012788772583 train acc: 0.99124 val loss: 0.24076818771362304 val acc: 0.9288 time: 1394.795580148697



epoch: 22 lr: 0.016 train loss: 0.030442327222824098 train acc: 0.99174 val loss: 0.24234764404296874 val acc: 0.9309 time: 1463.0227556228638



epoch: 23 lr: 0.008 train loss: 0.028436120920181274 train acc: 0.99256 val loss: 0.23991000442504884 val acc: 0.9301 time: 1531.7265470027924



epoch: 24 lr: 0.0 train loss: 0.027072189655303953 train acc: 0.99278 val loss: 0.24007553482055663 val acc: 0.9306 time: 1599.6102633476257


### LR with Gradual drop after One-CycleLR

In [0]:
model = DavidNet()
batches_per_epoch = len_train//BATCH_SIZE + 1

# lr_schedule = lambda t: np.interp([t], [0, (EPOCHS+1)//5, EPOCHS], [0, LEARNING_RATE, 0])[0]
lr_schedule = lambda t: np.interp([t], [0, (EPOCHS+1)//5, int(0.8*EPOCHS), EPOCHS], [0, LEARNING_RATE, 0.1*LEARNING_RATE, 0.009])[0]
global_step = tf.train.get_or_create_global_step()
lr_func = lambda: lr_schedule(global_step/batches_per_epoch)/BATCH_SIZE
opt = tf.train.MomentumOptimizer(lr_func, momentum=MOMENTUM, use_nesterov=True)
data_aug = lambda x, y: (tf.image.random_flip_left_right(tf.random_crop(x, [32, 32, 3])), y)

In [0]:
t = time.time()
test_set = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(BATCH_SIZE)

for epoch in range(EPOCHS):
  train_loss = test_loss = train_acc = test_acc = 0.0
  train_set = tf.data.Dataset.from_tensor_slices((x_train, y_train)).map(data_aug).shuffle(len_train).batch(BATCH_SIZE).prefetch(1)

  tf.keras.backend.set_learning_phase(1)
  for (x, y) in tqdm(train_set):
    with tf.GradientTape() as tape:
      loss, correct = model(x, y)

    var = model.trainable_variables
    grads = tape.gradient(loss, var)
    for g, v in zip(grads, var):
      g += v * WEIGHT_DECAY * BATCH_SIZE
    opt.apply_gradients(zip(grads, var), global_step=global_step)

    train_loss += loss.numpy()
    train_acc += correct.numpy()

  tf.keras.backend.set_learning_phase(0)
  for (x, y) in test_set:
    loss, correct = model(x, y)
    test_loss += loss.numpy()
    test_acc += correct.numpy()
    
  print('epoch:', epoch+1, 'lr:', lr_schedule(epoch+1), 'train loss:', train_loss / len_train, 'train acc:', train_acc / len_train, 'val loss:', test_loss / len_test, 'val acc:', test_acc / len_test, 'time:', time.time() - t)


epoch: 1 lr: 0.08 train loss: 1.5756401922607421 train acc: 0.4288 val loss: 1.072593603515625 val acc: 0.6101 time: 78.34312701225281



epoch: 2 lr: 0.16 train loss: 0.8382053173828125 train acc: 0.70194 val loss: 0.813119320678711 val acc: 0.7272 time: 143.32983422279358



epoch: 3 lr: 0.24 train loss: 0.6188879544067383 train acc: 0.78582 val loss: 1.0661950439453125 val acc: 0.6733 time: 208.1281681060791



epoch: 4 lr: 0.32 train loss: 0.540421672668457 train acc: 0.81242 val loss: 0.6837811279296875 val acc: 0.7789 time: 273.1088569164276



epoch: 5 lr: 0.4 train loss: 0.4646663754272461 train acc: 0.83982 val loss: 0.6191536453247071 val acc: 0.7981 time: 337.9119203090668


...


epoch: 20 lr: 0.033800000000000004 train loss: 0.03359828550338745 train acc: 0.99082 val loss: 0.2505376678466797 val acc: 0.9274 time: 1311.498485326767



epoch: 21 lr: 0.027600000000000006 train loss: 0.030669383087158202 train acc: 0.992 val loss: 0.2521941390991211 val acc: 0.9256 time: 1376.3941004276276



epoch: 22 lr: 0.021400000000000002 train loss: 0.028361185398101806 train acc: 0.99278 val loss: 0.2522179626464844 val acc: 0.9264 time: 1441.2829911708832



epoch: 23 lr: 0.015200000000000002 train loss: 0.026790784969329833 train acc: 0.99298 val loss: 0.25317908096313474 val acc: 0.9262 time: 1506.036226272583



epoch: 24 lr: 0.009 train loss: 0.026615504913330076 train acc: 0.9934 val loss: 0.2504736694335937 val acc: 0.9287 time: 1570.8621892929077


### Added Augmentation

In [0]:
# GlobalAvgPool
t = time.time()
test_set = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(BATCH_SIZE)

for epoch in range(EPOCHS):
  train_loss = test_loss = train_acc = test_acc = 0.0
  train_set = tf.data.Dataset.from_tensor_slices((x_train, y_train)).map(data_aug).shuffle(len_train).batch(BATCH_SIZE).prefetch(1)

  tf.keras.backend.set_learning_phase(1)
  for (x, y) in tqdm(train_set):
    with tf.GradientTape() as tape:
      loss, correct = model(x, y)

    var = model.trainable_variables
    grads = tape.gradient(loss, var)
    for g, v in zip(grads, var):
      g += v * WEIGHT_DECAY * BATCH_SIZE
    opt.apply_gradients(zip(grads, var), global_step=global_step)

    train_loss += loss.numpy()
    train_acc += correct.numpy()

  tf.keras.backend.set_learning_phase(0)
  for (x, y) in test_set:
    loss, correct = model(x, y)
    test_loss += loss.numpy()
    test_acc += correct.numpy()
    
  print('epoch:', epoch+1, 'lr:', lr_schedule(epoch+1), 'train loss:', train_loss / len_train, 'train acc:', train_acc / len_train, 'val loss:', test_loss / len_test, 'val acc:', test_acc / len_test, 'time:', time.time() - t)


epoch: 1 lr: 0.08 train loss: 1.5032733990478515 train acc: 0.45242 val loss: 1.0886321594238282 val acc: 0.6188 time: 111.50436234474182



epoch: 2 lr: 0.16 train loss: 0.8530325842285156 train acc: 0.69548 val loss: 0.9529444549560547 val acc: 0.6794 time: 199.9768569469452



epoch: 3 lr: 0.24 train loss: 0.6533779327392578 train acc: 0.77246 val loss: 0.7283566802978516 val acc: 0.7649 time: 288.34793758392334



epoch: 4 lr: 0.32 train loss: 0.5501688134765625 train acc: 0.80898 val loss: 0.9207446472167968 val acc: 0.7194 time: 376.78658080101013



epoch: 5 lr: 0.4 train loss: 0.5000022366333008 train acc: 0.82862 val loss: 0.774743960571289 val acc: 0.7654 time: 465.24572944641113


...


epoch: 20 lr: 0.03300000000000001 train loss: 0.039510198631286624 train acc: 0.9875 val loss: 0.2763801441192627 val acc: 0.9228 time: 1800.6386814117432



epoch: 21 lr: 0.026000000000000002 train loss: 0.03616086312294006 train acc: 0.98838 val loss: 0.2785358768463135 val acc: 0.9226 time: 1890.3359158039093



epoch: 22 lr: 0.019000000000000003 train loss: 0.03245591863632202 train acc: 0.98996 val loss: 0.2790522232055664 val acc: 0.9242 time: 1979.8410873413086



epoch: 23 lr: 0.012 train loss: 0.030312863445281984 train acc: 0.99022 val loss: 0.27781903839111327 val acc: 0.9227 time: 2069.5606100559235



epoch: 24 lr: 0.005 train loss: 0.027942822637557985 train acc: 0.99184 val loss: 0.27924166564941405 val acc: 0.9236 time: 2159.18039393425


### Added Augmentation

In [0]:
# GlobalAvgPool
t = time.time()
test_set = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(BATCH_SIZE)

for epoch in range(EPOCHS):
  train_loss = test_loss = train_acc = test_acc = 0.0
  train_set = tf.data.Dataset.from_tensor_slices((x_train, y_train)).map(data_aug).shuffle(len_train).batch(BATCH_SIZE).prefetch(1)

  tf.keras.backend.set_learning_phase(1)
  for (x, y) in tqdm(train_set):
    with tf.GradientTape() as tape:
      loss, correct = model(x, y)

    var = model.trainable_variables
    grads = tape.gradient(loss, var)
    for g, v in zip(grads, var):
      g += v * WEIGHT_DECAY * BATCH_SIZE
    opt.apply_gradients(zip(grads, var), global_step=global_step)

    train_loss += loss.numpy()
    train_acc += correct.numpy()

  tf.keras.backend.set_learning_phase(0)
  for (x, y) in test_set:
    loss, correct = model(x, y)
    test_loss += loss.numpy()
    test_acc += correct.numpy()
    
  print('epoch:', epoch+1, 'lr:', lr_schedule(epoch+1), 'train loss:', train_loss / len_train, 'train acc:', train_acc / len_train, 'val loss:', test_loss / len_test, 'val acc:', test_acc / len_test, 'time:', time.time() - t)


epoch: 1 lr: 0.08 train loss: 1.709021552734375 train acc: 0.3895 val loss: 1.326011083984375 val acc: 0.5089 time: 87.73148798942566



epoch: 2 lr: 0.16 train loss: 1.1988032983398438 train acc: 0.57196 val loss: 1.2408642211914063 val acc: 0.5604 time: 175.50283885002136



epoch: 3 lr: 0.24 train loss: 0.952559672241211 train acc: 0.66252 val loss: 0.8901464141845703 val acc: 0.685 time: 263.3326745033264



epoch: 4 lr: 0.32 train loss: 0.8045873928833008 train acc: 0.7156 val loss: 0.8165519332885742 val acc: 0.7088 time: 351.06829285621643



epoch: 5 lr: 0.4 train loss: 0.6980540258789063 train acc: 0.755 val loss: 0.8282282669067382 val acc: 0.707 time: 438.72689962387085


...


epoch: 20 lr: 0.03300000000000001 train loss: 0.2347546627807617 train acc: 0.91928 val loss: 0.49122921142578124 val acc: 0.8374 time: 1755.69038772583



epoch: 21 lr: 0.026000000000000002 train loss: 0.22667024085998536 train acc: 0.92264 val loss: 0.42604240493774415 val acc: 0.8596 time: 1843.4422998428345



epoch: 22 lr: 0.019000000000000003 train loss: 0.21167442672729492 train acc: 0.92786 val loss: 0.5472040420532227 val acc: 0.8263 time: 1931.0193729400635



epoch: 23 lr: 0.012 train loss: 0.20096443725585939 train acc: 0.93186 val loss: 0.4120720420837402 val acc: 0.866 time: 2018.7962889671326



epoch: 24 lr: 0.005 train loss: 0.19094788940429688 train acc: 0.93478 val loss: 0.4139376693725586 val acc: 0.8664 time: 2106.876798391342


### Backup